---

# Unir (Merge) Datasets

Notebook para unir los datasets de:
- Clima
- Demanda
- Factores socioeconómicos y poblacionales
- PIB
- Servicios públicos

**Dependencias:**
- [holidays](https://pypi.org/project/holidays/)
- [jupyter](https://jupyter.org/)
- [numpy](https://numpy.org/)
- [pandas](https://pandas.pydata.org/)

**Datasets Requeridos:**
- **dataset_clima.csv :** Dataset del clima generado por el Script correspondiente.
- **dataset_pib.csv :** Dataset del PIB generado por el Script correspondiente.
- **dataset_poblacion.csv :** Dataset de los factores socioeconómicos y poblacionales generado por el Script correspondiente.
- **dataset_servicios.csv :** Dataset de los servicios públicos generado por el Script correspondiente.
- **dataset_tebsa.csv :** Dataset de TEBSA a actualizar con los nuevos datos.
- **demanda.xlsx :** Archivo excel de demanda.

In [2]:
import datetime
import holidays
import numpy as np
import os
import pandas as pd
import warnings

In [3]:
warnings.filterwarnings('ignore')

---

## Constantes

### Directorio de Datos

Se puede modificar la variable `DATA_DIR_NAME` por el nombre deseado para el directorio de los datos.

In [4]:
DATA_DIR_NAME = 'data'
PROJECT_DIR = os.path.abspath(os.pardir)
DATA_DIR = os.path.join(PROJECT_DIR, DATA_DIR_NAME)

### Rango de Fechas
- `OLD_DATE` $\longrightarrow$ Fecha Inicial de los datos del dataset de TEBSA
- `START_DATE` $\longrightarrow$ Fecha Inicial de los datos nuevos a ser insertados
- `END_DATE` $\longrightarrow$ Fecha Final de los datos nuevos a ser insertados

In [3]:
OLD_DATE = '2017-01-01'
START_DATE = '2020-10-01'
END_DATE = '2022-03-31'

OLD = datetime.datetime.strptime(OLD_DATE, '%Y-%m-%d').date()
START = datetime.datetime.strptime(START_DATE, '%Y-%m-%d').date()
END = datetime.datetime.strptime(END_DATE, '%Y-%m-%d').date()

### Nombre de los Archivos

Estos nombres deben coincidir con los nombres de los archivos guardados en el directorio `<root_dir>/input_merge`.

In [ ]:
DATASET_NAMES = {
    'CLIMA': 'dataset_clima.csv',
    'DEMANDA': 'demanda.xlsx',
    'PIB': 'dataset_pib.csv',
    'POBLACION': 'dataset_poblacion.csv',
    'SERVICIOS': 'dataset_servicios.csv',
    'TEBSA': 'dataset_tebsa.csv'
}

### Opciones para la Limpieza de los Datos de los Archivos

Opciones para filtrar, renombrar, eliminar, reemplazar y reindexar los datos.

In [ ]:
DATASET_OPTIONS = {
    'CLIMA': {
        'to_drop': None,
        'to_filter_leq': (START, END),
        'to_filter_le': None,
        'to_reindex': None,
        'to_rename': None,
        'to_replace': {
            'narino': 'nariño'
        }
    },
    
    'DEMANDA': {
        'to_filter_leq': (START, END),
        'to_rename': {
            'Fecha': 'FECHA', 
            'Hora': 'HORA'
        }
    },
    
    'NEW': {
        'to_drop': None,
        'to_filter_leq': None,
        'to_filter_le': None,
        'to_reindex': [
            'FECHA',
            'HORA',
            'REGION',
            'PRESION',
            'HUMEDAD',
            'PRECIPITACION',
            'TEMPERATURA',
            'SENSACION_TERMICA',
            'PIB_REGIONAL',
            'POB_TOT',
            'POB_EDAD_T',
            'POB_ECO_ACT',
            'OCUPADOS',
            'DESOCUPADOS',
            'ESTRATO_1_AGUA',
            'ESTRATO_2_AGUA',
            'ESTRATO_3_AGUA',
            'ESTRATO_4_AGUA',
            'ESTRATO_5_AGUA',
            'ESTRATO_6_AGUA',
            'INDUSTRIAL_AGUA',
            'COMERCIAL_AGUA',
            'OFICIAL_AGUA',
            'OTROS_AGUA',
            'ESTRATO_1_LUZ',
            'ESTRATO_2_LUZ',
            'ESTRATO_3_LUZ',
            'ESTRATO_4_LUZ',
            'ESTRATO_5_LUZ',
            'ESTRATO_6_LUZ',
            'INDUSTRIAL_LUZ',
            'COMERCIAL_LUZ',
            'OFICIAL_LUZ',
            'OTROS_LUZ',
            'DAY_WEEK',
            'IS_HOLIDAY',
            'DEMANDA'
        ],
        'to_rename': None,
        'to_replace': None
    },
    
    'PIB': {
        'to_drop': [
            'AAAA', 
            'MM', 
            'DD', 
            'FECHA_ORD'
        ],
        'to_filter_leq': (START, END),
        'to_filter_le': None,
        'to_reindex': None,
        'to_rename': {
            'HH': 'HORA', 
            'PIB': 'PIB_REGIONAL'
        },
        'to_replace': None
    },
    
    'POBLACION': {
        'to_drop': None,
        'to_filter_leq': (START, END),
        'to_filter_le': None,
        'to_reindex': None,
        'to_rename': None,
        'to_replace': {
            'narino': 'nariño'
        }
    },
    
    'SERVICIOS': {
        'to_drop': None,
        'to_filter_leq': (START, END),
        'to_filter_le': None,
        'to_reindex': None,
        'to_rename': {
            'ACUEDUCTO_ESTRATO_1': 'ESTRATO_1_AGUA',
            'ACUEDUCTO_ESTRATO_2': 'ESTRATO_2_AGUA',
            'ACUEDUCTO_ESTRATO_3': 'ESTRATO_3_AGUA',
            'ACUEDUCTO_ESTRATO_4': 'ESTRATO_4_AGUA',
            'ACUEDUCTO_ESTRATO_5': 'ESTRATO_5_AGUA',
            'ACUEDUCTO_ESTRATO_6': 'ESTRATO_6_AGUA',
            'ACUEDUCTO_INDUSTRIAL': 'INDUSTRIAL_AGUA',
            'ACUEDUCTO_COMERCIAL': 'COMERCIAL_AGUA',
            'ACUEDUCTO_OFICIAL': 'OFICIAL_AGUA',
            'ACUEDUCTO_OTROS': 'OTROS_AGUA',
            'ENERGIA_ESTRATO_1': 'ESTRATO_1_LUZ',
            'ENERGIA_ESTRATO_2': 'ESTRATO_2_LUZ',
            'ENERGIA_ESTRATO_3': 'ESTRATO_3_LUZ',
            'ENERGIA_ESTRATO_4': 'ESTRATO_4_LUZ',
            'ENERGIA_ESTRATO_5': 'ESTRATO_5_LUZ',
            'ENERGIA_ESTRATO_6': 'ESTRATO_6_LUZ',
            'ENERGIA_INDUSTRIAL': 'INDUSTRIAL_LUZ',
            'ENERGIA_COMERCIAL': 'COMERCIAL_LUZ',
            'ENERGIA_OFICIAL': 'OFICIAL_LUZ',
            'ENERGIA_OTROS': 'OTROS_LUZ',
        },
        'to_replace': {
            'narino': 'nariño'
        }
    },
    
    'TEBSA': {
        'to_drop': [
            'TEMPERATURA_HUMEDA', 
            'HUMEDAD_RELATIVA_CALCULADA', 
            'SENSACION_TERMICA_CALIENTE'
        ],
        'to_filter_leq': None,
        'to_filter_le': (OLD, START),
        'to_reindex': [
            'FECHA',
            'HORA',
            'REGION',
            'PRESION',
            'HUMEDAD',
            'PRECIPITACION',
            'TEMPERATURA',
            'SENSACION_TERMICA',
            'PIB_REGIONAL',
            'POB_TOT',
            'POB_EDAD_T',
            'POB_ECO_ACT',
            'OCUPADOS',
            'DESOCUPADOS',
            'ESTRATO_1_AGUA',
            'ESTRATO_2_AGUA',
            'ESTRATO_3_AGUA',
            'ESTRATO_4_AGUA',
            'ESTRATO_5_AGUA',
            'ESTRATO_6_AGUA',
            'INDUSTRIAL_AGUA',
            'COMERCIAL_AGUA',
            'OFICIAL_AGUA',
            'OTROS_AGUA',
            'ESTRATO_1_LUZ',
            'ESTRATO_2_LUZ',
            'ESTRATO_3_LUZ',
            'ESTRATO_4_LUZ',
            'ESTRATO_5_LUZ',
            'ESTRATO_6_LUZ',
            'INDUSTRIAL_LUZ',
            'COMERCIAL_LUZ',
            'OFICIAL_LUZ',
            'OTROS_LUZ',
            'DAY_WEEK',
            'IS_HOLIDAY',
            'DEMANDA'
        ],
        'to_rename': {
            'TEMPERATURA_SECA': 'TEMPERATURA', 
            'SENSACION_TERMICA_IDEAM': 'SENSACION_TERMICA'
        },
        'to_replace': None
    }
}

---

## Creación del Árbol de Directorios

- \< *root_dir* \>
  - input_merge
    - *dataset_clima.csv*
    - *dataset_pib.csv*
    - *dataset_poblacion.csv*
    - *dataset_servicios.csv*
    - *dataset_tebsa.csv*
    - *demanda.xlsx*
  - output_merge
    - **dataset_tebsa_{OLD_DATE}_{END_DATE}.csv**
    - **dataset_tebsa_{OLD_DATE}_{END_DATE}_fixed.csv**

In [5]:
''' Generar Arbol de Directorios
Genera el árbol de directorios en los que se alojaran los datos necesarios para el correcto funcionamiento 
del script, así como el directorio para los datos de salida.

@param root_dir: Directorio raiz para el arbol de directorios
@param input_: Flag para la creación del directorio de los datos de entrada (Valor default = True)
@param output_: Flag para la creación del directorio de los datos de salida (Valor default = True) 
'''
def generate_directory_tree(root_dir, input_=True, output_=True):
    if input_:
        os.makedirs(os.path.join(root_dir, 'input_merge'), exist_ok=True)
    
    if output_:
        os.makedirs(os.path.join(root_dir, 'output_merge'), exist_ok=True)

In [ ]:
generate_directory_tree(DATA_DIR)

---

## Funciones

**Funciones de Verificación:**
- `has_nan`
- `equal_len`
- `check_zeros_departs`

**Funciones para Datasets:**
- `load_dataset`
- `process_dataset`
- `process_demanda`
- `merge_datasets`
- `generate_weekday_holidays`

**Funciones de Ajuste:**
- `get_temporary_variables`
- `fix_zeros`

### Funciones de Verificación

In [ ]:
''' Verificar valores nulos
Verifica si el dataframe contiene valores nulos.

@param df: Dataframe a verificar
@return: True si el dataframe contiene valores nulos
@return: False si el dataframe no contiene valores nulos
'''
def has_nan(df):
    return df.isna().sum().sum() > 0


''' Comparar cantidad de filas
Verifica si el dataframe tiene misma cantidad de filas que el valor enviado.

@param df: Dataframe a verificar
@param nrows: Número de filas a verificar
@return: True si el dataframe tiene la misma cantidad de filas que nrows
@return: False si el dataframe no tiene la misma cantidad de filas que nrows
'''
def equal_len(df, nrows):
    return len(df) == nrows


def check_zeros_departs(df):
    great_consumers = [
        'cerromatoso',
        'cirainfanta',
        'drummond',
        'emec',
        'intercor',
        'oxy',
        'rubiales',
        'tuboscaribe'
    ]

    reg_zeros = (df.loc[df['DEMANDA'] == 0]['REGION'].value_counts().to_dict().keys())

    for reg in reg_zeros:
        if reg not in great_consumers:
            return False
    
    return True

### Funciones para Datasets

In [ ]:

def load_dataset(name, sep=',', target='csv', sheet_name=0, input_=True, to_rename=None, usecols=None):
    dir_mode = 'input_merge' if input_ else 'output_merge'
    
    if target == 'csv':
        df = pd.read_csv(os.path.join(DATA_DIR, dir_mode, name), sep=sep, usecols=usecols)
    elif target == 'xlsx':
        df = pd.read_excel(os.path.join(DATA_DIR, dir_mode, name), sheet_name=sheet_name)

    if to_rename is not None:
        df.rename(columns=to_rename, inplace=True)

    df['FECHA'] = pd.to_datetime(df['FECHA'], infer_datetime_format=True)
    df['FECHA'] = df['FECHA'].dt.date
    
    return df


def process_dataset(df, options={}):
    df['REGION'] = df['REGION'].apply(lambda x: x.lower())
    df = df[df['REGION'] != 'cundinamarca']

    if options.get('to_filter_leq') is not None:
        df = df[(df['FECHA'] >= to_filter_leq[0]) & (df['FECHA'] <= to_filter_leq[1])]

    if options.get('to_filter_le') is not None:
        df = df[(df['FECHA'] >= to_filter_le[0]) & (df['FECHA'] < to_filter_le[1])]

    if options.get('to_replace') is not None:
        df.replace(to_replace=to_replace, inplace=True)

    if options.get('to_rename') is not None:
        df.rename(columns=to_rename, inplace=True)

    if options.get('to_drop') is not None:
        df.drop(columns=to_drop, inplace=True)

    df.sort_values(by=['REGION', 'FECHA', 'HORA'], inplace=True)

    if options.get('to_reindex') is not None:
        df = df.reindex(columns=to_reindex)

    return df


def process_demanda(df, id_vars, to_filter_leq=None, var_name='REGION', value_name='DEMANDA'):
    if to_filter_leq is not None:
        df = df[(df['FECHA'] >= to_filter_leq[0]) & (df['FECHA'] <= to_filter_leq[1])]

    df = pd.melt(df, id_vars=id_vars, var_name=var_name, value_name=value_name)

    df['REGION'] = df['REGION'].apply(lambda x: x.lower())
    df = df[df['REGION'] != 'cundinamarca']

    df.sort_values(by=['REGION', 'FECHA', 'HORA'], inplace=True)

    return df


def merge_dataframes(df_1, df_2, on, how='outer'):
    df = pd.merge(df_1, df_2, how=how, on=on)
    return df


def generate_weekday_holidays(df):
    holidays_range = {
        x.strftime('%Y-%m-%d') for x in holidays.CountryHoliday(
            'COL',
            years=set(pd.date_range(
                start=START_DATE,
                end=END_DATE,
                freq='D').year
            ), 
            observed=True
        )
    }

    df['DAY_WEEK'] = pd.to_datetime(df['FECHA'], infer_datetime_format=True).map(lambda x: x.dayofweek)
    df['IS_HOLIDAY'] = df['FECHA'].map(lambda x: 1 if str(x) in holidays_range else 0)
    
    return df

## Funciones de Ajuste

In [ ]:
def get_temporary_variables(df):
    df['DAY_TYPE'] = [0] * len(df)
    df.loc[((df['IS_HOLIDAY'] == 1) | (df['DAY_WEEK'] == 6)), 'DAY_TYPE'] = 2
    df.loc[((df['DAY_WEEK'] == 5) & (df['DAY_TYPE'] == 0)), 'DAY_TYPE'] = 1

    df['YEAR'] = df['FECHA'].apply(lambda x: x.year)
    df['MONTH'] = df['FECHA'].apply(lambda x: x.month)

    return df


def fix_zeros(df, df_group, df_zeros):
    df = df.loc[df['DEMANDA'] != 0]

    values_fill = []
    for index, row in df_zeros.iterrows():
        values_fill.append(
            df_group.loc[(
                (df_group['REGION'] == row['REGION']) & 
                (df_group['YEAR'] == row['YEAR']) & 
                (df_group['MONTH'] == row['MONTH']) & 
                (df_group['DAY_TYPE'] == row['DAY_TYPE']) & 
                (df_group['HORA'] == row['HORA'])
            )]['DEMANDA'].values[0]
        )

    df_zeros['DEMANDA'] = values_fill

    df = pd.concat([df, df_zeros])
    
    df.sort_values(by=['REGION', 'FECHA', 'HORA'], inplace=True)

    return df

---

## Cargar y Procesar Datasets

### Clima

In [ ]:
df_clima = load_dataset(DATASET_NAMES['CLIMA'])
df_clima = process_dataset(df_clima, options=DATASET_OPTIONS['CLIMA'])
df_clima

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA
0,2020-10-01,1,antioquia,840.033333,88.215943,14.805,16.990926,18.713188
1,2020-10-01,2,antioquia,842.750000,88.718752,26.300,17.038800,18.488753
2,2020-10-01,3,antioquia,842.125000,90.841265,8.955,16.834266,17.644681
3,2020-10-01,4,antioquia,839.300000,93.334961,20.504,16.460470,16.610112
4,2020-10-01,5,antioquia,842.425000,93.536953,34.805,16.401083,16.532308
...,...,...,...,...,...,...,...,...
498859,2022-03-31,20,tulua,832.350000,99.000000,2.300,17.800000,13.863864
498860,2022-03-31,21,tulua,821.050000,99.000000,0.200,17.933333,13.915338
498861,2022-03-31,22,tulua,821.800000,99.000000,0.000,17.866667,13.888839
498862,2022-03-31,23,tulua,822.350000,98.000000,0.000,17.600000,14.285036


#### Verificación

In [ ]:
nrows = len(df_clima)

In [ ]:
assert not has_nan(df_clima)
assert equal_len(df_clima, nrows)

### PIB

In [ ]:
df_pib = load_dataset(DATASET_NAMES['PIB'], sep=';')
df_pib = process_dataset(df_pib, options=DATASET_OPTIONS['PIB'])
df_pib

,HORA,FECHA,REGION,PIB_REGIONAL
0,1,2020-10-01,antioquia,32705.152057
1,2,2020-10-01,antioquia,32705.152057
2,3,2020-10-01,antioquia,32705.152057
3,4,2020-10-01,antioquia,32705.152057
4,5,2020-10-01,antioquia,32705.152057
...,...,...,...,...
498859,20,2022-03-31,tulua,22835.859989
498860,21,2022-03-31,tulua,22835.859989
498861,22,2022-03-31,tulua,22835.859989
498862,23,2022-03-31,tulua,22835.859989


#### Verificación

In [ ]:
assert not has_nan(df_pib)
assert equal_len(df_pib, nrows)

### Población

In [ ]:
df_poblacion = load_dataset(DATASET_NAMES['POBLACION'])
df_poblacion = process_dataset(df_poblacion, options=DATASET_OPTIONS['POBLACION'])
df_poblacion

,FECHA,HORA,REGION,POB_TOT,POB_EDAD_T,POB_ECO_ACT,OCUPADOS,DESOCUPADOS
433224,2020-10-01,1,antioquia,6825.407226,5618.305094,3684.085674,3239.587453,383.520558
433225,2020-10-01,2,antioquia,6825.407226,5618.305094,3684.085674,3239.587453,383.520558
433226,2020-10-01,3,antioquia,6825.407226,5618.305094,3684.085674,3239.587453,383.520558
433227,2020-10-01,4,antioquia,6825.407226,5618.305094,3684.085674,3239.587453,383.520558
433228,2020-10-01,5,antioquia,6825.407226,5618.305094,3684.085674,3239.587453,383.520558
...,...,...,...,...,...,...,...,...
91891,2022-03-31,20,tulua,4899.187447,4085.062564,2710.435377,2403.783609,322.675089
91892,2022-03-31,21,tulua,4899.187447,4085.062564,2710.435377,2403.783609,322.675089
91893,2022-03-31,22,tulua,4899.187447,4085.062564,2710.435377,2403.783609,322.675089
91894,2022-03-31,23,tulua,4899.187447,4085.062564,2710.435377,2403.783609,322.675089


#### Verificación

In [ ]:
assert not has_nan(df_poblacion)
assert equal_len(df_poblacion, nrows)

### Servicios

In [ ]:
df_servicios = load_dataset(DATASET_NAMES['SERVICIOS'])
df_servicios = process_dataset(df_servicios, options=DATASET_OPTIONS['SERVICIOS'])
df_servicios

,FECHA,HORA,REGION,ESTRATO_1_LUZ,ESTRATO_2_LUZ,ESTRATO_3_LUZ,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,...,ESTRATO_1_AGUA,ESTRATO_2_AGUA,ESTRATO_3_AGUA,ESTRATO_4_AGUA,ESTRATO_5_AGUA,ESTRATO_6_AGUA,INDUSTRIAL_AGUA,COMERCIAL_AGUA,OFICIAL_AGUA,OTROS_AGUA
111264,2020-10-01,1,antioquia,56038087.0,103957383.0,85694803.0,30569281.0,20699980.0,11472327.0,227410826.0,...,2520504.0,6772288.0,6169266.0,1954201.0,1175168.0,600456.0,1009766.0,1380696.0,670037.0,238121.0
111302,2020-10-01,2,antioquia,56038087.0,103957383.0,85694803.0,30569281.0,20699980.0,11472327.0,227410826.0,...,2520504.0,6772288.0,6169266.0,1954201.0,1175168.0,600456.0,1009766.0,1380696.0,670037.0,238121.0
111340,2020-10-01,3,antioquia,56038087.0,103957383.0,85694803.0,30569281.0,20699980.0,11472327.0,227410826.0,...,2520504.0,6772288.0,6169266.0,1954201.0,1175168.0,600456.0,1009766.0,1380696.0,670037.0,238121.0
111378,2020-10-01,4,antioquia,56038087.0,103957383.0,85694803.0,30569281.0,20699980.0,11472327.0,227410826.0,...,2520504.0,6772288.0,6169266.0,1954201.0,1175168.0,600456.0,1009766.0,1380696.0,670037.0,238121.0
111416,2020-10-01,5,antioquia,56038087.0,103957383.0,85694803.0,30569281.0,20699980.0,11472327.0,227410826.0,...,2520504.0,6772288.0,6169266.0,1954201.0,1175168.0,600456.0,1009766.0,1380696.0,670037.0,238121.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610123,2022-03-31,20,tulua,77991554.0,66909373.0,74706745.0,17739500.0,12732913.0,6954671.0,91089711.0,...,2549610.0,5321196.0,3724749.0,1369245.0,955149.0,383405.0,399688.0,1354987.0,642969.0,250347.0
610124,2022-03-31,21,tulua,77991554.0,66909373.0,74706745.0,17739500.0,12732913.0,6954671.0,91089711.0,...,2549610.0,5321196.0,3724749.0,1369245.0,955149.0,383405.0,399688.0,1354987.0,642969.0,250347.0
610125,2022-03-31,22,tulua,77991554.0,66909373.0,74706745.0,17739500.0,12732913.0,6954671.0,91089711.0,...,2549610.0,5321196.0,3724749.0,1369245.0,955149.0,383405.0,399688.0,1354987.0,642969.0,250347.0
610126,2022-03-31,23,tulua,77991554.0,66909373.0,74706745.0,17739500.0,12732913.0,6954671.0,91089711.0,...,2549610.0,5321196.0,3724749.0,1369245.0,955149.0,383405.0,399688.0,1354987.0,642969.0,250347.0


#### Verificación

In [ ]:
assert not has_nan(df_servicios)
assert equal_len(df_servicios, nrows)

### TEBSA

In [ ]:
df_tebsa = load_dataset(DATASET_NAMES['TEBSA'])
df_tebsa = process_dataset(df_tebsa, options=DATASET_OPTIONS['TEBSA'])
df_tebsa

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA
837285,2017-01-01,1,antioquia,877.2000,93.87500,0.00000,23.868589,22.953191,29679.423075,6534.494269,...,145230.000000,87753.000000,37684.000000,8959.000000,107738.000000,5703.000000,2256.000000,6,1,629.88000
197339,2017-01-01,2,antioquia,876.7000,94.25000,0.00000,23.883650,22.945803,29679.423075,6534.494269,...,145230.000000,87753.000000,37684.000000,8959.000000,107738.000000,5703.000000,2256.000000,6,1,583.18000
347769,2017-01-01,3,antioquia,875.9750,94.87500,0.00000,20.414349,17.326933,29679.423075,6534.494269,...,145230.000000,87753.000000,37684.000000,8959.000000,107738.000000,5703.000000,2256.000000,6,1,552.80000
478794,2017-01-01,4,antioquia,875.4000,95.00000,0.00000,19.727817,16.656013,29679.423075,6534.494269,...,145230.000000,87753.000000,37684.000000,8959.000000,107738.000000,5703.000000,2256.000000,6,1,530.52000
443422,2017-01-01,5,antioquia,875.6250,95.62500,0.00000,19.167719,16.018144,29679.423075,6534.494269,...,145230.000000,87753.000000,37684.000000,8959.000000,107738.000000,5703.000000,2256.000000,6,1,516.12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612335,2020-09-30,20,tulua,805.3734,74.67447,158.88939,19.908480,22.314834,22290.709463,4830.137196,...,103081.441922,67915.246603,20092.823628,2057.246624,53299.055026,1619.743339,1478.973885,2,0,35.20889
886423,2020-09-30,21,tulua,806.0580,75.95777,161.25142,19.908480,22.050222,22290.709463,4830.137196,...,103081.441922,67915.246603,20092.823628,2057.246624,53299.055026,1619.743339,1478.973885,2,0,33.28157
884995,2020-09-30,22,tulua,806.0580,76.67953,163.61096,19.252527,21.891540,22290.709463,4830.137196,...,103081.441922,67915.246603,20092.823628,2057.246624,53299.055026,1619.743339,1478.973885,2,0,31.85419
617027,2020-09-30,23,tulua,806.0580,77.43757,166.25032,18.745886,21.776374,22290.709463,4830.137196,...,103081.441922,67915.246603,20092.823628,2057.246624,53299.055026,1619.743339,1478.973885,2,0,28.34865


#### Verificación

In [ ]:
assert not has_nan(df_tebsa)
assert equal_len(df_tebsa, (START - OLD).days * 24 * 37)

### Demanda

In [ ]:
df_demanda = load_dataset(
    DATASET_NAMES['DEMANDA'], 
    target='xlsx', 
    sheet_name='Real', 
    to_rename=DATASET_OPTIONS['DEMANDA']['to_rename']
)

df_demanda = process_demanda(
    df_demanda, 
    list(DATASET_OPTIONS['DEMANDA']['to_rename'].values()),
    to_filter_leq=DATASET_OPTIONS['DEMANDA']['to_filter_leq']
)

df_demanda

,FECHA,HORA,REGION,DEMANDA
45984,2017-01-01,1,antioquia,629.88000
45985,2017-01-01,2,antioquia,583.18000
45986,2017-01-01,3,antioquia,552.80000
45987,2017-01-01,4,antioquia,530.52000
45988,2017-01-01,5,antioquia,516.12000
...,...,...,...,...
1701403,2022-03-31,20,tulua,34.89881
1701404,2022-03-31,21,tulua,33.42721
1701405,2022-03-31,22,tulua,30.96817
1701406,2022-03-31,23,tulua,27.46424


#### Verificación

In [ ]:
assert not has_nan(df_demanda)
assert equal_len(df_demanda, nrows)

#### Verificación de Valores 0

In [ ]:
df_demanda[df_demanda['DEMANDA'] == 0]

,FECHA,HORA,REGION,DEMANDA
894440,2019-05-15,9,arauca,0.0
894441,2019-05-15,10,arauca,0.0
894442,2019-05-15,11,arauca,0.0
894443,2019-05-15,12,arauca,0.0
894444,2019-05-15,13,arauca,0.0
...,...,...,...,...
1746781,2022-03-06,14,tuboscaribe,0.0
1746782,2022-03-06,15,tuboscaribe,0.0
1746783,2022-03-06,16,tuboscaribe,0.0
1746784,2022-03-06,17,tuboscaribe,0.0


In [ ]:
df_demanda[df_demanda['DEMANDA'] == 0]['REGION'].value_counts()

emec            2124
putumayo         351
oxy              282
drummond         229
santander        216
guaviare         187
bajoputumayo     119
cirainfanta       99
huila             48
arauca            41
tuboscaribe       21
caqueta           10
choco              8
intercor           4
cerromatoso        3
meta               2
boyaca             1
Name: REGION, dtype: int64

---

## Merge Datasets

In [ ]:
df_new = merge_dataframes(
    df_clima, 
    merge_dataframes(
        df_pib, 
        merge_dataframes(
            df_poblacion, 
            merge_dataframes(
                df_servicios,
                df_demanda,
                on=['FECHA', 'HORA', 'REGION']
            ),
            on=['FECHA', 'HORA', 'REGION']
        ),
        on=['FECHA', 'HORA', 'REGION']
    ),
    on=['FECHA', 'HORA', 'REGION']
)

df_new

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,ESTRATO_2_AGUA,ESTRATO_3_AGUA,ESTRATO_4_AGUA,ESTRATO_5_AGUA,ESTRATO_6_AGUA,INDUSTRIAL_AGUA,COMERCIAL_AGUA,OFICIAL_AGUA,OTROS_AGUA,DEMANDA
0,2020-10-01,1,antioquia,840.033333,88.215943,14.805,16.990926,18.713188,32705.152057,6825.407226,...,6772288.0,6169266.0,1954201.0,1175168.0,600456.0,1009766.0,1380696.0,670037.0,238121.0,841.06405
1,2020-10-01,2,antioquia,842.750000,88.718752,26.300,17.038800,18.488753,32705.152057,6825.407226,...,6772288.0,6169266.0,1954201.0,1175168.0,600456.0,1009766.0,1380696.0,670037.0,238121.0,779.63663
2,2020-10-01,3,antioquia,842.125000,90.841265,8.955,16.834266,17.644681,32705.152057,6825.407226,...,6772288.0,6169266.0,1954201.0,1175168.0,600456.0,1009766.0,1380696.0,670037.0,238121.0,771.37198
3,2020-10-01,4,antioquia,839.300000,93.334961,20.504,16.460470,16.610112,32705.152057,6825.407226,...,6772288.0,6169266.0,1954201.0,1175168.0,600456.0,1009766.0,1380696.0,670037.0,238121.0,778.30255
4,2020-10-01,5,antioquia,842.425000,93.536953,34.805,16.401083,16.532308,32705.152057,6825.407226,...,6772288.0,6169266.0,1954201.0,1175168.0,600456.0,1009766.0,1380696.0,670037.0,238121.0,819.61315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485731,2022-03-31,20,tulua,832.350000,99.000000,2.300,17.800000,13.863864,22835.859989,4899.187447,...,5321196.0,3724749.0,1369245.0,955149.0,383405.0,399688.0,1354987.0,642969.0,250347.0,34.89881
485732,2022-03-31,21,tulua,821.050000,99.000000,0.200,17.933333,13.915338,22835.859989,4899.187447,...,5321196.0,3724749.0,1369245.0,955149.0,383405.0,399688.0,1354987.0,642969.0,250347.0,33.42721
485733,2022-03-31,22,tulua,821.800000,99.000000,0.000,17.866667,13.888839,22835.859989,4899.187447,...,5321196.0,3724749.0,1369245.0,955149.0,383405.0,399688.0,1354987.0,642969.0,250347.0,30.96817
485734,2022-03-31,23,tulua,822.350000,98.000000,0.000,17.600000,14.285036,22835.859989,4899.187447,...,5321196.0,3724749.0,1369245.0,955149.0,383405.0,399688.0,1354987.0,642969.0,250347.0,27.46424


#### Verificación

In [ ]:
assert not has_nan(df_new)
assert equal_len(df_new, nrows)

---

## Generar Dias de la Semana y Festivos

In [ ]:
df_new = generate_weekday_holidays(df_new)
df_new = process_dataset(df_new, options=DATASET_OPTIONS['NEW'])
df_new

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA
0,2020-10-01,1,antioquia,840.033333,88.215943,14.805,16.990926,18.713188,32705.152057,6825.407226,...,30569281.0,20699980.0,11472327.0,227410826.0,141589968.0,24969609.0,17957368.0,3,0,841.06405
1,2020-10-01,2,antioquia,842.750000,88.718752,26.300,17.038800,18.488753,32705.152057,6825.407226,...,30569281.0,20699980.0,11472327.0,227410826.0,141589968.0,24969609.0,17957368.0,3,0,779.63663
2,2020-10-01,3,antioquia,842.125000,90.841265,8.955,16.834266,17.644681,32705.152057,6825.407226,...,30569281.0,20699980.0,11472327.0,227410826.0,141589968.0,24969609.0,17957368.0,3,0,771.37198
3,2020-10-01,4,antioquia,839.300000,93.334961,20.504,16.460470,16.610112,32705.152057,6825.407226,...,30569281.0,20699980.0,11472327.0,227410826.0,141589968.0,24969609.0,17957368.0,3,0,778.30255
4,2020-10-01,5,antioquia,842.425000,93.536953,34.805,16.401083,16.532308,32705.152057,6825.407226,...,30569281.0,20699980.0,11472327.0,227410826.0,141589968.0,24969609.0,17957368.0,3,0,819.61315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485731,2022-03-31,20,tulua,832.350000,99.000000,2.300,17.800000,13.863864,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,34.89881
485732,2022-03-31,21,tulua,821.050000,99.000000,0.200,17.933333,13.915338,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,33.42721
485733,2022-03-31,22,tulua,821.800000,99.000000,0.000,17.866667,13.888839,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,30.96817
485734,2022-03-31,23,tulua,822.350000,98.000000,0.000,17.600000,14.285036,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,27.46424


#### Verificación

In [ ]:
assert not has_nan(df_new)
assert equal_len(df_new, nrows)
assert equal_len(df_new.columns, len(df_tebsa.columns))

---

## Concatenar Datasets

In [ ]:
df_final = pd.concat([df_tebsa, df_new])
df_final = process_dataset(df_final)
df_final

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA
837285,2017-01-01,1,antioquia,877.200,93.875,0.0,23.868589,22.953191,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,629.88000
197339,2017-01-01,2,antioquia,876.700,94.250,0.0,23.883650,22.945803,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,583.18000
347769,2017-01-01,3,antioquia,875.975,94.875,0.0,20.414349,17.326933,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,552.80000
478794,2017-01-01,4,antioquia,875.400,95.000,0.0,19.727817,16.656013,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,530.52000
443422,2017-01-01,5,antioquia,875.625,95.625,0.0,19.167719,16.018144,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,516.12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485731,2022-03-31,20,tulua,832.350,99.000,2.3,17.800000,13.863864,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,34.89881
485732,2022-03-31,21,tulua,821.050,99.000,0.2,17.933333,13.915338,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,33.42721
485733,2022-03-31,22,tulua,821.800,99.000,0.0,17.866667,13.888839,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,30.96817
485734,2022-03-31,23,tulua,822.350,98.000,0.0,17.600000,14.285036,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,27.46424


#### Verificación

In [ ]:
assert not has_nan(df_final)
assert equal_len(df_final, (START - OLD).days * 24 * 37 + nrows)
assert equal_len(df_final.columns, len(df_tebsa.columns))

---

## Guardar Dataset Final

In [ ]:
df_final.to_csv(os.path.join(DATA_DIR, 'output_merge', f'dataset_tebsa_{OLD_DATE}_{END_DATE}.csv'), index=False, header=True)

---

## Verificar Dataset Final

In [5]:
df = load_dataset(f'dataset_tebsa_{OLD_DATE}_{END_DATE}.csv', input_=False)
df

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA
0,2017-01-01,1,antioquia,877.200,93.875,0.0,23.868589,22.953191,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,629.88000
1,2017-01-01,2,antioquia,876.700,94.250,0.0,23.883650,22.945803,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,583.18000
2,2017-01-01,3,antioquia,875.975,94.875,0.0,20.414349,17.326933,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,552.80000
3,2017-01-01,4,antioquia,875.400,95.000,0.0,19.727817,16.656013,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,530.52000
4,2017-01-01,5,antioquia,875.625,95.625,0.0,19.167719,16.018144,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,516.12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701403,2022-03-31,20,tulua,832.350,99.000,2.3,17.800000,13.863864,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,34.89881
1701404,2022-03-31,21,tulua,821.050,99.000,0.2,17.933333,13.915338,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,33.42721
1701405,2022-03-31,22,tulua,821.800,99.000,0.0,17.866667,13.888839,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,30.96817
1701406,2022-03-31,23,tulua,822.350,98.000,0.0,17.600000,14.285036,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,27.46424


#### Verificación

In [ ]:
assert not has_nan(df)
assert equal_len(df, (START - OLD).days * 24 * 37 + nrows)
assert equal_len(df.columns, len(df_final.columns))

#### Verificación Valores 0

In [6]:
df.loc[df['DEMANDA'] == 0]

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA
66728,2019-05-15,9,arauca,992.30,86.0,0.0,23.540225,23.185957,1083.087858,271.232186,...,278.0,0.0,0.0,17.0,1825.0,454.0,0.0,2,0,0.0
66729,2019-05-15,10,arauca,992.70,83.0,0.1,24.376526,24.746828,1083.087858,271.232186,...,278.0,0.0,0.0,17.0,1825.0,454.0,0.0,2,0,0.0
66730,2019-05-15,11,arauca,992.60,80.0,0.0,23.728324,24.003174,1083.087858,271.232186,...,278.0,0.0,0.0,17.0,1825.0,454.0,0.0,2,0,0.0
66731,2019-05-15,12,arauca,991.90,75.0,0.0,24.492588,25.262956,1083.087858,271.232186,...,278.0,0.0,0.0,17.0,1825.0,454.0,0.0,2,0,0.0
66732,2019-05-15,13,arauca,990.60,70.0,0.0,24.861374,25.821502,1083.087858,271.232186,...,278.0,0.0,0.0,17.0,1825.0,454.0,0.0,2,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654813,2022-03-06,14,tuboscaribe,1007.10,63.0,0.0,31.750000,37.389887,8353.813500,2229.166317,...,7758564.0,4529286.0,8010206.0,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0
1654814,2022-03-06,15,tuboscaribe,1005.75,66.5,0.0,31.250000,37.202054,8353.813500,2229.166317,...,7758564.0,4529286.0,8010206.0,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0
1654815,2022-03-06,16,tuboscaribe,1004.75,68.0,0.0,31.250000,37.654888,8353.813500,2229.166317,...,7758564.0,4529286.0,8010206.0,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0
1654816,2022-03-06,17,tuboscaribe,1004.50,69.0,0.0,30.600000,36.269642,8353.813500,2229.166317,...,7758564.0,4529286.0,8010206.0,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0


In [7]:
df.loc[df['DEMANDA'] == 0]['REGION'].value_counts()

emec            2124
putumayo         351
oxy              282
drummond         229
santander        216
guaviare         187
bajoputumayo     119
cirainfanta       99
huila             48
arauca            41
tuboscaribe       21
caqueta           10
choco              8
intercor           4
cerromatoso        3
meta               2
boyaca             1
Name: REGION, dtype: int64

In [8]:
print(
    'MIN', 
    min(df.loc[df['DEMANDA'] == 0]['FECHA']), 
    ', MAX', 
    max(df.loc[df['DEMANDA'] == 0]['FECHA'])
)

MIN 2017-02-15 , MAX 2022-03-24


In [9]:
df.loc[df['DEMANDA'] == 0]['FECHA'].value_counts()

2019-12-04    47
2019-12-03    46
2019-12-31    33
2020-01-23    33
2019-10-24    33
              ..
2021-06-18     1
2018-12-01     1
2018-12-09     1
2021-06-26     1
2017-05-12     1
Name: FECHA, Length: 263, dtype: int64

In [10]:
df.loc[((df['DEMANDA'] == 0) & (df['FECHA'] >= START))]

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA
85469,2021-07-04,6,arauca,996.65,98.0,0.1,23.45,21.536028,1270.446965,277.706322,...,107212.0,0.0,0.0,44796816.0,1898725.0,2381969.0,291632.0,6,0,0.0
85470,2021-07-04,7,arauca,997.35,98.0,0.0,23.45,21.536028,1270.446965,277.706322,...,107212.0,0.0,0.0,44796816.0,1898725.0,2381969.0,291632.0,6,0,0.0
85471,2021-07-04,8,arauca,997.80,95.5,0.0,24.35,23.882039,1270.446965,277.706322,...,107212.0,0.0,0.0,44796816.0,1898725.0,2381969.0,291632.0,6,0,0.0
85472,2021-07-04,9,arauca,998.35,92.0,0.0,25.20,26.077348,1270.446965,277.706322,...,107212.0,0.0,0.0,44796816.0,1898725.0,2381969.0,291632.0,6,0,0.0
85473,2021-07-04,10,arauca,998.80,85.0,0.0,26.10,28.074818,1270.446965,277.706322,...,107212.0,0.0,0.0,44796816.0,1898725.0,2381969.0,291632.0,6,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654813,2022-03-06,14,tuboscaribe,1007.10,63.0,0.0,31.75,37.389887,8353.813500,2229.166317,...,7758564.0,4529286.0,8010206.0,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0
1654814,2022-03-06,15,tuboscaribe,1005.75,66.5,0.0,31.25,37.202054,8353.813500,2229.166317,...,7758564.0,4529286.0,8010206.0,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0
1654815,2022-03-06,16,tuboscaribe,1004.75,68.0,0.0,31.25,37.654888,8353.813500,2229.166317,...,7758564.0,4529286.0,8010206.0,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0
1654816,2022-03-06,17,tuboscaribe,1004.50,69.0,0.0,30.60,36.269642,8353.813500,2229.166317,...,7758564.0,4529286.0,8010206.0,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0


In [11]:
df.loc[((df['DEMANDA'] == 0) & (df['FECHA'] >= START))]['REGION'].value_counts()

oxy             162
drummond         41
putumayo         35
guaviare         31
bajoputumayo     29
cirainfanta      28
arauca           23
tuboscaribe      19
Name: REGION, dtype: int64

In [12]:
print(
    'MIN', 
    min(df.loc[((df['DEMANDA'] == 0) & (df['FECHA'] >= START))]['FECHA']), 
    ', MAX', 
    max(df.loc[((df['DEMANDA'] == 0) & (df['FECHA'] >= START))]['FECHA'])
)

MIN 2020-10-11 , MAX 2022-03-24


In [13]:
df.loc[((df['DEMANDA'] == 0) & (df['FECHA'] >= START))]['FECHA'].value_counts()

2021-07-11    24
2021-07-15    24
2021-07-14    24
2021-07-13    24
2021-07-12    24
2021-01-31    23
2021-08-15    21
2021-07-16    18
2021-03-24    14
2021-03-25    14
2022-01-16    13
2022-03-23    12
2021-07-10    11
2022-03-24    11
2021-07-04    10
2022-03-06    10
2021-12-31     9
2021-01-10     9
2021-03-20     8
2020-12-31     8
2021-07-24     7
2021-06-19     6
2022-03-05     6
2021-01-24     5
2021-12-14     5
2021-12-12     4
2020-12-09     4
2021-07-26     3
2021-06-17     3
2020-10-18     3
2022-02-13     2
2021-07-07     2
2021-11-08     2
2022-02-14     1
2022-01-26     1
2021-06-26     1
2021-06-18     1
2020-10-11     1
Name: FECHA, dtype: int64

---

## Ajustar Dataset

In [15]:
TEMPORARY_COLUMNS = ['DAY_TYPE', 'YEAR', 'MONTH']
GROUP_COLUMNS = ['REGION', 'YEAR', 'MONTH', 'DAY_TYPE', 'HORA']

In [16]:
df = load_dataset(f'dataset_tebsa_{OLD_DATE}_{END_DATE}.csv', input_=False)

ncols, nrows = len(df.columns), len(df)

df = get_temporary_variables(df)
df

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA,DAY_TYPE,YEAR,MONTH
0,2017-01-01,1,antioquia,877.200,93.875,0.0,23.868589,22.953191,29679.423075,6534.494269,...,8959.0,107738.0,5703.0,2256.0,6,1,629.88000,2,2017,1
1,2017-01-01,2,antioquia,876.700,94.250,0.0,23.883650,22.945803,29679.423075,6534.494269,...,8959.0,107738.0,5703.0,2256.0,6,1,583.18000,2,2017,1
2,2017-01-01,3,antioquia,875.975,94.875,0.0,20.414349,17.326933,29679.423075,6534.494269,...,8959.0,107738.0,5703.0,2256.0,6,1,552.80000,2,2017,1
3,2017-01-01,4,antioquia,875.400,95.000,0.0,19.727817,16.656013,29679.423075,6534.494269,...,8959.0,107738.0,5703.0,2256.0,6,1,530.52000,2,2017,1
4,2017-01-01,5,antioquia,875.625,95.625,0.0,19.167719,16.018144,29679.423075,6534.494269,...,8959.0,107738.0,5703.0,2256.0,6,1,516.12000,2,2017,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701403,2022-03-31,20,tulua,832.350,99.000,2.3,17.800000,13.863864,22835.859989,4899.187447,...,91089711.0,106420075.0,12949809.0,16083305.0,3,0,34.89881,0,2022,3
1701404,2022-03-31,21,tulua,821.050,99.000,0.2,17.933333,13.915338,22835.859989,4899.187447,...,91089711.0,106420075.0,12949809.0,16083305.0,3,0,33.42721,0,2022,3
1701405,2022-03-31,22,tulua,821.800,99.000,0.0,17.866667,13.888839,22835.859989,4899.187447,...,91089711.0,106420075.0,12949809.0,16083305.0,3,0,30.96817,0,2022,3
1701406,2022-03-31,23,tulua,822.350,98.000,0.0,17.600000,14.285036,22835.859989,4899.187447,...,91089711.0,106420075.0,12949809.0,16083305.0,3,0,27.46424,0,2022,3


In [17]:
df_zeros = df.loc[df['DEMANDA'] == 0]
df_zeros

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA,DAY_TYPE,YEAR,MONTH
66728,2019-05-15,9,arauca,992.30,86.0,0.0,23.540225,23.185957,1083.087858,271.232186,...,17.0,1825.0,454.0,0.0,2,0,0.0,0,2019,5
66729,2019-05-15,10,arauca,992.70,83.0,0.1,24.376526,24.746828,1083.087858,271.232186,...,17.0,1825.0,454.0,0.0,2,0,0.0,0,2019,5
66730,2019-05-15,11,arauca,992.60,80.0,0.0,23.728324,24.003174,1083.087858,271.232186,...,17.0,1825.0,454.0,0.0,2,0,0.0,0,2019,5
66731,2019-05-15,12,arauca,991.90,75.0,0.0,24.492588,25.262956,1083.087858,271.232186,...,17.0,1825.0,454.0,0.0,2,0,0.0,0,2019,5
66732,2019-05-15,13,arauca,990.60,70.0,0.0,24.861374,25.821502,1083.087858,271.232186,...,17.0,1825.0,454.0,0.0,2,0,0.0,0,2019,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654813,2022-03-06,14,tuboscaribe,1007.10,63.0,0.0,31.750000,37.389887,8353.813500,2229.166317,...,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0,2,2022,3
1654814,2022-03-06,15,tuboscaribe,1005.75,66.5,0.0,31.250000,37.202054,8353.813500,2229.166317,...,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0,2,2022,3
1654815,2022-03-06,16,tuboscaribe,1004.75,68.0,0.0,31.250000,37.654888,8353.813500,2229.166317,...,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0,2,2022,3
1654816,2022-03-06,17,tuboscaribe,1004.50,69.0,0.0,30.600000,36.269642,8353.813500,2229.166317,...,63138220.0,45194861.0,6587829.0,4849815.0,6,0,0.0,2,2022,3


In [18]:
df_group = df.groupby(GROUP_COLUMNS).mean().reset_index()
df_group

,REGION,YEAR,MONTH,DAY_TYPE,HORA,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,...,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA
0,antioquia,2017,1,0,1,842.739048,89.409812,0.146000,23.545700,22.816885,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,1.952381,0.0,668.253333
1,antioquia,2017,1,0,2,842.320635,90.015013,0.111381,23.508033,22.675119,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,1.952381,0.0,637.640000
2,antioquia,2017,1,0,3,841.822698,90.918165,0.128571,19.531664,17.923749,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,1.952381,0.0,630.244762
3,antioquia,2017,1,0,4,841.531746,91.536380,0.100000,18.771989,17.393769,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,1.952381,0.0,642.429048
4,antioquia,2017,1,0,5,841.495556,91.648157,0.509524,18.211891,17.195875,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,1.952381,0.0,704.008095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167827,tulua,2022,3,2,20,833.308563,86.945427,4.547729,18.698029,18.982615,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,6.000000,0.0,29.379585
167828,tulua,2022,3,2,21,834.372650,88.422844,7.826371,18.271337,18.433469,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,6.000000,0.0,29.328420
167829,tulua,2022,3,2,22,835.703705,89.490074,10.426535,17.965281,18.028791,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,6.000000,0.0,27.826705
167830,tulua,2022,3,2,23,836.290985,90.347427,12.068639,17.707804,17.699224,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,6.000000,0.0,25.413585


In [19]:
df_fixed = fix_zeros(df, df_group, df_zeros)
df_fixed.drop(columns=TEMPORARY_COLUMNS, inplace=True)
df_fixed

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA
0,2017-01-01,1,antioquia,877.200,93.875,0.0,23.868589,22.953191,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,629.88000
1,2017-01-01,2,antioquia,876.700,94.250,0.0,23.883650,22.945803,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,583.18000
2,2017-01-01,3,antioquia,875.975,94.875,0.0,20.414349,17.326933,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,552.80000
3,2017-01-01,4,antioquia,875.400,95.000,0.0,19.727817,16.656013,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,530.52000
4,2017-01-01,5,antioquia,875.625,95.625,0.0,19.167719,16.018144,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,516.12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701403,2022-03-31,20,tulua,832.350,99.000,2.3,17.800000,13.863864,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,34.89881
1701404,2022-03-31,21,tulua,821.050,99.000,0.2,17.933333,13.915338,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,33.42721
1701405,2022-03-31,22,tulua,821.800,99.000,0.0,17.866667,13.888839,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,30.96817
1701406,2022-03-31,23,tulua,822.350,98.000,0.0,17.600000,14.285036,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,27.46424


In [20]:
df_fixed.loc[df_fixed['DEMANDA'] == 0]['REGION'].value_counts()

emec    1365
Name: REGION, dtype: int64

#### Verificación

In [21]:
assert not has_nan(df)
assert equal_len(df_fixed, nrows)
assert equal_len(df_fixed.columns, ncols)
assert check_zeros_departs(df_fixed)

---

## Guardar Dataset Ajustado

In [22]:
df_fixed.to_csv(os.path.join(DATA_DIR, 'output_merge', f'dataset_tebsa_{OLD_DATE}_{END_DATE}_fixed.csv'), index=False, header=True)

---

## Verificar Valores 0 Despues del Ajuste

In [23]:
df = load_dataset(os.path.join('final', f'dataset_tebsa_{OLD_DATE}_{END_DATE}_fixed.csv'))
df

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA
0,2017-01-01,1,antioquia,877.200,93.875,0.0,23.868589,22.953191,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,629.88000
1,2017-01-01,2,antioquia,876.700,94.250,0.0,23.883650,22.945803,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,583.18000
2,2017-01-01,3,antioquia,875.975,94.875,0.0,20.414349,17.326933,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,552.80000
3,2017-01-01,4,antioquia,875.400,95.000,0.0,19.727817,16.656013,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,530.52000
4,2017-01-01,5,antioquia,875.625,95.625,0.0,19.167719,16.018144,29679.423075,6534.494269,...,145230.0,87753.0,37684.0,8959.0,107738.0,5703.0,2256.0,6,1,516.12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701403,2022-03-31,20,tulua,832.350,99.000,2.3,17.800000,13.863864,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,34.89881
1701404,2022-03-31,21,tulua,821.050,99.000,0.2,17.933333,13.915338,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,33.42721
1701405,2022-03-31,22,tulua,821.800,99.000,0.0,17.866667,13.888839,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,30.96817
1701406,2022-03-31,23,tulua,822.350,98.000,0.0,17.600000,14.285036,22835.859989,4899.187447,...,17739500.0,12732913.0,6954671.0,91089711.0,106420075.0,12949809.0,16083305.0,3,0,27.46424


In [24]:
df.loc[df['DEMANDA'] == 0]

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA,PIB_REGIONAL,POB_TOT,...,ESTRATO_4_LUZ,ESTRATO_5_LUZ,ESTRATO_6_LUZ,INDUSTRIAL_LUZ,COMERCIAL_LUZ,OFICIAL_LUZ,OTROS_LUZ,DAY_WEEK,IS_HOLIDAY,DEMANDA
897768,2019-10-01,1,emec,801.408333,94.529630,7.610,15.069197,16.750801,3345.552905,1820.595115,...,10325.0,3675.0,10.0,1108.0,11254.0,691.0,329.0,1,0,0.0
897769,2019-10-01,2,emec,800.891667,94.945667,7.100,15.319628,16.345289,3345.552905,1820.595115,...,10325.0,3675.0,10.0,1108.0,11254.0,691.0,329.0,1,0,0.0
897770,2019-10-01,3,emec,800.491667,93.813243,1.690,12.099839,20.900506,3345.552905,1820.595115,...,10325.0,3675.0,10.0,1108.0,11254.0,691.0,329.0,1,0,0.0
897771,2019-10-01,4,emec,800.200000,93.849822,8.600,11.453183,22.077282,3345.552905,1820.595115,...,10325.0,3675.0,10.0,1108.0,11254.0,691.0,329.0,1,0,0.0
897772,2019-10-01,5,emec,800.358333,93.094323,3.859,11.540845,22.395910,3345.552905,1820.595115,...,10325.0,3675.0,10.0,1108.0,11254.0,691.0,329.0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899967,2019-12-31,16,emec,759.345455,79.571232,8.905,18.958897,21.189432,3369.661242,1825.662016,...,9999.0,3546.0,1.0,989.0,9966.0,491.0,328.0,1,0,0.0
899972,2019-12-31,21,emec,761.081818,94.750000,3.800,13.283887,18.483971,3369.661242,1825.662016,...,9999.0,3546.0,1.0,989.0,9966.0,491.0,328.0,1,0,0.0
899973,2019-12-31,22,emec,761.790909,94.666667,9.000,13.213385,18.630498,3369.661242,1825.662016,...,9999.0,3546.0,1.0,989.0,9966.0,491.0,328.0,1,0,0.0
899974,2019-12-31,23,emec,762.245455,94.666667,2.400,13.294385,18.521425,3369.661242,1825.662016,...,9999.0,3546.0,1.0,989.0,9966.0,491.0,328.0,1,0,0.0


In [25]:
df.loc[df['DEMANDA'] == 0]['REGION'].value_counts()

emec    1365
Name: REGION, dtype: int64

In [26]:
print(
    'MIN', 
    min(df.loc[df['DEMANDA'] == 0]['FECHA']), 
    ', MAX', 
    max(df.loc[df['DEMANDA'] == 0]['FECHA'])
)

MIN 2019-10-01 , MAX 2019-12-31


In [27]:
df.loc[df['DEMANDA'] == 0]['FECHA'].value_counts()

2019-10-19    24
2019-10-05    24
2019-10-06    24
2019-10-27    24
2019-10-26    24
              ..
2019-11-08    10
2019-11-07    10
2019-11-06    10
2019-11-05    10
2019-11-01    10
Name: FECHA, Length: 92, dtype: int64